In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.SILENT)

In [2]:
print('This example illustrates a layered network with a loop.')

model = LayeredNetwork('myLayeredModel')

# Layer 1 - Reference task
P1 = Processor(model, 'P1', GlobalConstants.MaxInt, SchedStrategy.INF)
T1 = Task(model, 'T1', 1, SchedStrategy.REF).on(P1)
T1.set_think_time(Immediate())
E1 = Entry(model, 'Entry').on(T1)

# Layer 2 - Service task
P2 = Processor(model, 'P2', GlobalConstants.MaxInt, SchedStrategy.INF)
T2 = Task(model, 'T2', 1, SchedStrategy.INF).on(P2).set_think_time(Immediate())
E2 = Entry(model, 'E2').on(T2)

# Layer 3 - Backend task
P3 = Processor(model, 'P3', 5, SchedStrategy.PS)
T3 = Task(model, 'T3', 20, SchedStrategy.INF).on(P3)
T3.set_think_time(Exp.fit_mean(10))
E3 = Entry(model, 'E1').on(T3)  # Note: Entry named 'E1' in original

# Activities for T1 (with loop)
A1 = Activity(model, 'A1', Exp.fit_mean(1)).on(T1).bound_to(E1)
A2 = Activity(model, 'A2', Exp.fit_mean(2)).on(T1)
A3 = Activity(model, 'A3', Exp.fit_mean(3)).on(T1).synch_call(E2, 1)

# Activities for T2 (with fork-join)
B1 = Activity(model, 'B1', Exp.fit_mean(0.1)).on(T2).bound_to(E2)
B2 = Activity(model, 'B2', Exp.fit_mean(0.2)).on(T2)
B3 = Activity(model, 'B3', Exp.fit_mean(0.3)).on(T2)
B4 = Activity(model, 'B4', Exp.fit_mean(0.4)).on(T2)
B5 = Activity(model, 'B5', Exp.fit_mean(0.5)).on(T2)
B6 = Activity(model, 'B6', Exp.fit_mean(0.6)).on(T2).synch_call(E3, 1).replies_to(E2)

# Activities for T3 (with or-fork/join)
C1 = Activity(model, 'C1', Exp.fit_mean(0.1)).on(T3).bound_to(E3)
C2 = Activity(model, 'C2', Exp.fit_mean(0.2)).on(T3)
C3 = Activity(model, 'C3', Exp.fit_mean(0.3)).on(T3)
C4 = Activity(model, 'C4', Exp.fit_mean(0.4)).on(T3)
C5 = Activity(model, 'C5', Exp.fit_mean(0.5)).on(T3).replies_to(E3)

This example illustrates a layered network with a loop.


In [3]:
# Add precedence relationships

# T1: Loop with 3 iterations
T1.add_precedence(ActivityPrecedence.Loop(A1, [A2, A3], 3))

# T2: Serial connection and AND fork-join
T2.add_precedence(ActivityPrecedence.Serial(B4, B5))
T2.add_precedence(ActivityPrecedence.AndFork(B1, [B2, B3, B4]))
T2.add_precedence(ActivityPrecedence.AndJoin([B2, B3, B5], B6))

# T3: OR fork-join with probabilities
T3.add_precedence(ActivityPrecedence.OrFork(C1, [C2, C3, C4], [0.3, 0.3, 0.4]))
T3.add_precedence(ActivityPrecedence.OrJoin([C2, C3, C4], C5))

In [4]:
# Solve with different solvers

if LQNS.isAvailable():
    solver_lqns = LQNS(model)
    avg_table_lqns = solver_lqns.avg_table()

 Node   QLen   Util   RespT   Tput
   P1 0.0000 0.7958  0.0000 0.0000
   T1 1.0000 0.7958  0.0000 0.0796
Entry 1.0000 0.7958 12.5667 0.0796
   A1 0.0796 0.0796  1.0000 0.0796
   A2 0.4775 0.4775  2.0000 0.2387
   A3 0.4430 0.2387  5.5667 0.0796
   P2 0.0000 0.1671  0.0000 0.0000
   T2 0.2042 0.1671  0.0000 0.0796
   E2 0.2042 0.1671  2.5667 0.0796
   B1 0.0080 0.0080  0.1000 0.0796
   B2 0.0159 0.0159  0.2000 0.0796
   B3 0.0239 0.0239  0.3000 0.0796
   B4 0.0318 0.0318  0.4000 0.0796
   B5 0.0398 0.0398  0.5000 0.0796
   B6 0.1201 0.0477  1.5098 0.0796
   P3 0.0000 0.0724  0.0000 0.0000
   T3 0.0724 0.0724  0.0000 0.0796
   E1 0.0724 0.0724  0.9098 0.0796
   C1 0.0080 0.0080  0.1000 0.0796
   C2 0.0048 0.0048  0.2000 0.0239
   C3 0.0072 0.0072  0.2999 0.0239
   C4 0.0127 0.0127  0.3999 0.0318
   C5 0.0398 0.0398  0.4999 0.0796


In [5]:
solver_ln = LN(model)
avg_table_ln = solver_ln.avg_table()

 Node  NodeType   QLen       Util        RespT       ResidT       ArvR       Tput
   P1 Processor 1.0000 1.0000e-06 1000000.0000 1000000.0000 1.0000e-06 1.0000e-06
   P2 Processor 0.9997 9.9973e-07  100000.0000  100000.0000 9.9973e-06 9.9973e-06
   P3 Processor 1.1458 1.8854e-01       0.6077       0.6077 1.8854e+00 1.8854e+00
   T2      Task 0.5236 5.2356e-01       2.1000       2.1000 2.4931e-01 2.4931e-01
   T3      Task 0.5236 2.6178e-02      30.0000      30.0000 1.7452e-02 1.7452e-02
Entry     Entry 1.0000 1.0000e-06 1000000.0000 1000000.0000 1.0000e-06 1.0000e-06
   E2     Entry 0.9997 9.9973e-07  100000.0000  100000.0000 9.9973e-06 9.9973e-06
   E1     Entry 1.1458 1.8854e-01       0.6077       0.6077 1.8854e+00 1.8854e+00
